# Structured Streamig(Socket Source)

In [1]:
lines=spark.readStream \
.format("socket") \
.option('host','localhost') \
.option('port',9999) \
.load()

In [2]:
from pyspark.sql.functions import split,explode

In [5]:
words=lines.select(explode(split(lines.value," ")).alias('word'))
wordCounts=words.groupBy('word').count()
wordCounts

DataFrame[word: string, count: bigint]

In [8]:
socket_streaming=wordCounts.writeStream \
.format('console') \
.outputMode('complete') \
.start()
socket_streaming.awaitTermination()

KeyboardInterrupt: 

# Spark Structured Streaming(File Source)

In [1]:
from pyspark.sql.functions import StringType
from pyspark.sql.types import StructField,StructType,TimestampType

In [2]:
schema=StructType([StructField("Date",TimestampType(),False),StructField("Msg",StringType(),False)])

In [3]:
streamDF=spark.readStream \
.option('delimiter','|') \
.schema(schema) \
.csv("file:///home/sanjay/SparkTutorial/data/logs") \

In [4]:
OutDF=streamDF.createOrReplaceTempView("StDF")
OutDF1 = spark.sql("select * from StDF")

In [25]:
OutDF1.writeStream \
.format('console') \
.outputMode('append') \
.start() \
.awaitTermination()

KeyboardInterrupt: 

# Aggregations on Structured Streaming

In [5]:
OutDF2 = spark.sql("select Date,count(Msg) from StDF group by Date")

In [31]:
OutDF2.writeStream \
.format('console') \
.outputMode('update') \
.start() \
.awaitTermination()

KeyboardInterrupt: 

# Foreach Sink

In [6]:
OutDF3 = spark.sql("select Msg from StDF")

In [16]:
def process(row):
    print(row[0]+"_GP")

In [17]:
OutDF3.writeStream \
.foreach(process) \
.outputMode('append') \
.start() \
.awaitTermination()

KeyboardInterrupt: 